In [1]:
import mlflow
import pandas as pd
import numpy as np
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
from src.models.model import MatrixFactorization
from src.data.preprocessing import load_ml1m_data, preprocess_ratings, split_data
from src.data.dataset import RecommenderDataset
from src.training.trainer import train_model
from torch.utils.data import DataLoader

In [2]:
hyperparameter_grind = {
    'embedding_dim' : [50, 100, 150],
    'reg_lambda' : [0.001, 0.01, 0.1],
    'dropout' : [0.1, 0.2, 0.3]
}

In [3]:
def prepare_data(config):

    ratings_df, _ = load_ml1m_data('../data/raw/ml-1m')
    processed_df, user_mapping, item_mapping = preprocess_ratings(ratings_df)
    train_data, val_data = split_data(processed_df)

    train_dataset = RecommenderDataset(train_data)
    val_dataset = RecommenderDataset(val_data)

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=True)
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=False
    )

    return train_loader, val_loader, len(user_mapping), len(item_mapping)

In [4]:
def run_hyperparameter_experiment(config, hyperparams, train_loader, val_loader):

    model = MatrixFactorization(
        num_users=config['num_users'],
        n_items=config['n_items'],
        embedding_dim=hyperparams['embedding_dim'],
        reg_lambda=hyperparams['reg_lambda'],
    )

    trained_model = train_model(model, train_loader, val_loader, config)

    client = mlflow.tracking.MlflowClient()
    current_run = mlflow.active_run()
    metrics = client.get_run(current_run.info.run_id).data.metrics

    epochs = range(config['training']['num_epochs'])
    train_losses = [metrics.get(f'train_loss_{i}', 0) for i in epochs]
    val_losses = [metrics.get(f'val_loss_{i}', 0) for i in epochs]

    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='TrainLoss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Loss Curves (dim={hyperparams["embedding_dim"]}, lambda={hyperparams["reg_lambda"]}, dropout={hyperparams["dropout"]})')
    plt.legend()
    plt.grid(True)

    plot_path = f"mlruns/0/{current_run.info.run_id}/loss_plot_{hyperparams['embedding_dim']}_{hyperparams['reg_lambda']}_{hyperparams['dropout']}.png"

    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()

    return {
        'model': trained_model,
        'params': hyperparams,
        'train_losses': train_losses,
        'val_losses': val_losses
    }


In [5]:
def run_experiments(config):

    train_loader, val_loader, num_users, n_items = prepare_data(config)

    config.update({
        'num_users': num_users,
        'n_items': n_items
    })

    results = []

    try:
        mlflow.end_run()
    except:
        pass

    with mlflow.start_run(run_name="Hyperparameter_optimization") as parent_run:
        for params in product(*hyperparameter_grind.values()):
            hyperparams = dict(zip(hyperparameter_grind.keys(), params))
            print(f"Running experiment with parameters: {hyperparams}")

            with mlflow.start_run(nested=True) as child_run:

                mlflow.log_params({
                    "embedding_dim": hyperparams['embedding_dim'],
                    "reg_lambda": hyperparams['reg_lambda'],
                    "dropout": hyperparams['dropout'],
                    "batch_size": config['training']['batch_size'],
                    "learning_rate": config['training']['learning_rate'],
                    "num_epochs": config['training']['num_epochs']
                })

                model = run_hyperparameter_experiment(
                    config,
                    hyperparams,
                    train_loader,
                    val_loader
                )

                mlflow.log_params(hyperparams)

                results.append({
                    'params': hyperparams,
                    'model': model,
                })

        return results

In [6]:
if __name__ == '__main__':
    with open('../config/config.yaml', 'r') as f:
        config = yaml.safe_load(f)

    results = run_experiments(config)

Running experiment with parameters: {'embedding_dim': 50, 'reg_lambda': 0.001, 'dropout': 0.1}
Epoch 1/15 
Train Loss: 95.05388364508077
Val Loss: 0.8647248819136726
----------------------------------------
Epoch 2/15 
Train Loss: 0.8388159650712455
Val Loss: 0.832718297367247
----------------------------------------
Epoch 3/15 
Train Loss: 0.8212985032764919
Val Loss: 0.8281270281881837
----------------------------------------
Epoch 4/15 
Train Loss: 0.8174495252620542
Val Loss: 0.8275914346407182
----------------------------------------
Epoch 5/15 
Train Loss: 0.8163062832116756
Val Loss: 0.8275716483249774
----------------------------------------
Epoch 6/15 
Train Loss: 0.8158085202585361
Val Loss: 0.8274392001640698
----------------------------------------
Epoch 7/15 
Train Loss: 0.8155343811468898
Val Loss: 0.8278301787763471
----------------------------------------
Epoch 8/15 
Train Loss: 0.8154869116988743
Val Loss: 0.8279825413953549
----------------------------------------
Epo

KeyboardInterrupt: 